<a href="https://colab.research.google.com/github/g150446/colab/blob/main/light_enja2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日->英 翻訳器をつくります．

まずは必要なデータやライブラリをそろえます．

In [1]:
! git clone https://github.com/nymwa/light_enja2.git
! mv light_enja2/* .
! pip install -r requirements.txt

Cloning into 'light_enja2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 4), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (30/30), 3.18 MiB | 4.25 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.0


データに対して前処理を行います

ダウンロードした時点でデータセットは単語分割やトークン化がすでに施されています．
マージ数8000のBPEでサブワードに分割します．

In [17]:
! bash preproc.sh

+ TRAIN_EN=corpus/train.en
+ TRAIN_JA=corpus/train.ja
+ VALID_EN=corpus/valid.en
+ VALID_JA=corpus/valid.ja
+ TEST_EN=corpus/test.en
+ cat corpus/train.en corpus/train.ja
+ python src/learn.py --input train.enja --prefix bpe --vocab-size 4000 --character-coverage 0.9995 --threads 1
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 1
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extrem

学習を行います．

少ないデータでかつかなり軽量なモデルでの学習ですが，10分ぐらいかかります．

In [18]:
!fairseq-train \
    data-bin \
    --save-interval 10 \
    --max-epoch 10 \
    --update-freq 1 \
    --max-tokens 4000 \
    --arch transformer \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 1024 \
    --encoder-attention-heads 4 \
    --encoder-layers 4 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 1024 \
    --decoder-attention-heads 4 \
    --decoder-layers 4 \
    --share-all-embeddings \
    --dropout 0.3 \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.002 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 2000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.01 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.6

2023-07-07 04:56:29.519076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-07 04:56:30 | INFO | numexpr.utils | NumExpr defaulting to 4 threads.
2023-07-07 04:56:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-07-07 04:56:33 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale':

テストデータで翻訳を行い，その性能を評価します．

評価尺度BLEUで30ぐらい出ると思います．

In [19]:
! fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint10.pt \
    --beam 5 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu corpus/test.ja

2023-07-07 05:10:17.944290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-07 05:10:19 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-07-07 05:10:21 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0,

翻訳結果と正解データの冒頭10文です．

In [20]:
! head output.txt corpus/test.ja

==> output.txt <==
彼女 の 母 は 美しい 女性 です 。
その 畑 に は 6 羊 が い た 。
今朝 は 散歩 の 間 、 とても 写真 を 撮る こと が でき た 。
今晩 電話 し ます 。
彼女 は 本 の 虫 だ 。
式 は 終わっ た 。
私 は 彼 に 前日 を 楽しん だ か どう か 尋ね た 。
彼 は 慎重 で は ない 。
列車 は いつも 時間どおり に し て いる 。
2 時 に 出発 し たら 、 彼 ら は 6 時 に 着く べき だ 。

==> corpus/test.ja <==
彼女 の お母さん は きれい な 女 の 人 です 。
野原 に は 六 頭 の 羊 が い た 。
この 朝 の 散歩 で とても 素晴らしい 写真 を 撮る こと が でき た 。
今夜 電話 し ます 。
彼女 は いわゆる 本 の 虫 です 。
式典 は 終わり まし た 。
昨日 は 楽しかっ た か 、 と 私 は 彼 に 尋ね た 。
彼 に は 心配 事 が ない 。
その 列車 は いつも 時刻 通り だ 。
2 時 に 出発 し たら 、 6 時 に は つく はず だ 。


In [14]:
!pip install -U sacremoses
!pip install sentencepiece
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 73.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11171284 sha256=5074681487f0209ce13af77f67f8b7e79f6d4fc53d229656b93d8bbe08112ea8
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=1

たのしい

In [15]:
import re
import unicodedata
from sacremoses import MosesTokenizer
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

In [21]:
mt = MosesTokenizer(lang = 'en')
sp = spm.SentencePieceProcessor(model_file='bpe.model')
model = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint10.pt', data_name_or_path='data-bin')

def preproc_en(x):
  x = unicodedata.normalize('NFKC', x)
  x = re.sub(mt.AGGRESSIVE_HYPHEN_SPLIT[0], r'\1 - ', x)
  x = mt.tokenize(x, escape = False)
  x = ' '.join(x)
  x = x.lower()
  x = ' '.join(sp.encode(x, out_type = 'str'))
  return x

def translate(x):
  x = preproc_en(x)
  x = model.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

In [22]:
while True:
  x = input('英文を入力 > ')
  if not x:
    break
  x = translate(x)
  print('翻訳結果 > ' + x)

英文を入力 > What is cause of pneumonia?


RuntimeError: ignored